## Data Downloader for ERA5 and GLoFAS data

defines some methods to download era5 data

In [1]:
import cdsapi

In [ ]:
def era5_retrieval(filename, base_levels='pressure', area='50/7/47/20' form='netcdf', levels='700'):
    # download era5 data with the cdsapi
    
    
    
    c = cdsapi.Client()
    c.retrieve(f"reanalysis-era5-{base_levels}-levels", {
            "product_type":   "reanalysis",
            "format":         form,
            "area":           area, # N/W/S/E
            "variable":       ["geopotential","temperature"],
            "pressure_level": "700",
            "year":           "2017",
            "month":          "01",
            "day":            ["01", "02", "03", "04",
                               "05", "06", "07", "08",
                               "09", "10", "11", "12",
                               "13", "14", "15", "16",
                               "17", "18", "19", "20",
                               "21", "22", "23", "24",
                               "25", "26", "27", "28",
                               "29", "30", "31"],
            "time":           ["00", "01", "02", "03", "04", "05",
                              "06", "07", "08", "09", "10", "11",
                              "12", "13","14", "15", "16", "17",
                              "18", "19", "20","21", "22", "23"]
        }, "example_era5.nc")
    return True

reanalysis-era5-pressure-levels


In [ ]:
time_start = 20170501
time_end = 20170731




